In [ ]:
import pygmt
import numpy as np

# projection center and size of the Earth
lon0, lat0, size = 170, 30, 15

fig = pygmt.Figure()
# plotting the earth relief
fig.grdimage("@earth_relief_15m", projection=f"G{lon0}/{lat0}/{size}c", shading=True, cmap="geo")

# cut 1/8 of the Earth
x=[130.0, 130.0, 220.0]
y=[90.0, 0.0, 0.0]
fig.plot(x=x, y=y, color="darkgray", pen="0.5p,white@50")

# plot Moho
scale = 6300/6371
newsize = size * scale
xshift = yshift = size * (1 - scale) / 2.0
fig.plot(x=x, y=y, color="162/99/57", pen="0.5p,white@50", projection=f"G{lon0}/{lat0}/{newsize}c", xshift=f"a{xshift}c", yshift=f"a{yshift}c")

# plot CMB
scale = 3480/6371
newsize = size * scale
xshift = yshift = size * (1 - scale) / 2.0
fig.plot(x=x, y=y, color="lightgreen", pen="0.5p,white@50", projection=f"G{lon0}/{lat0}/{newsize}c", xshift=f"a{xshift}c", yshift=f"a{yshift}c")

#### Here, the map projection is switched to linear projection!
# The following codes are used to plot the XYZ axis
# mapproject is not wrapped in PyGMT
with pygmt.helpers.GMTTempFile() as outfile:
    with pygmt.clib.Session() as lib:
        file_context = lib.virtualfile_from_vectors(x, y)
        with file_context as fname:
            lib.call_module("mapproject", f"{fname} -JG{lon0}/{lat0}/{size}c -Rg > {outfile.name}")
        newx, newy = outfile.loadtxt(unpack=True)
for x0, y0 in zip(newx, newy):
    fig.plot(x=[x0, size*0.5], y=[y0, size*0.5], projection="x1c", region=[0, size, 0, size], pen="1p,white@80")

# plot the inner core
scale = 1221/6371
fig.image("3Dball.eps", position=f"JMC+w{size*scale}c")

# Add more labels
fig.text(x=size*0.5, y=size*0.5, text="Inner Core", font="12p,1,white", offset="0.1c/0c")

fig.savefig("3DEarth.jpg")
fig.show()